In [1]:
### taking close look at One Image
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [2]:
import os, random, warnings
import plotly.express as px
import warnings
import numpy as np
import pandas as pd

from glob import glob


import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
# import plotly.express as px
import lib.benchmark_func as func

/home/wujiayi/.conda/envs/tf2-gpu/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


## Notebook for benchmark image features
We are finding the set of features that are the best by recursively adding features and sort by the average cv5 result.


In [3]:
def stepwise(add_class, ini_classes, best_res, refit='Accuracy',\
             parameters = {'kernel':[ 'poly'], 'degree':[3],'coef0':np.arange(-0.5, 0.5, 0.1),\
                     'C': list(range(1, 5)), 'gamma':np.arange(0.1, 1.5, 0.1)}):
    
    for i in add_class:
        X, y = np.array(tr[ini_classes+[i]]), np.array(tr['decay'])
        X = sklearn.preprocessing.scale(X) 
        y = func.str2int(y)

        results = func.SVC_GS(X, y, refit=refit,parameters = parameters)
        if results.best_score_ > best_res:
            best_res = results.best_score_
            ini_classes.append(i)
            print(i)
        
    return ini_classes, best_res

## import all the feature datas

In [4]:
#GLCM
GLCMdf = pd.read_csv('Data/tr_EF_512_loc_pivot.csv', index_col = 0)#.set_index('pid')
GLCMclass = list(GLCMdf.keys())
overalldf = pd.read_csv('Data/tr_EF_512.csv', index_col = 0)#.set_index('pid')
#OD
adf = pd.read_csv('Data/pivot_area_b0_0602_aug_all_12120p8.csv', index_col = 0)
odclass = list(adf.keys()[:20])
adf = adf[odclass]

In [5]:
non_overalls = []
overalls = []
for i in GLCMclass:
    if 'overall' in i:
        overalls.append(i)
    else:
        non_overalls.append(i)

In [6]:
# percentage as a target 
jdf = pd.read_csv('Data/percentage_classes.csv', index_col = 0)
jdf

,slope,intercept,r_value,p_value,std_err,nor,decay
ID00007637202177411956430,-0.079696,54.716059,-0.640030,0.063366,0.036162,-0.075740,slow
ID00009637202177434476278,-0.218565,89.052698,-0.868881,0.002359,0.047065,-0.127626,fast
ID00010637202177584971671,-0.458238,89.043063,-0.893802,0.001157,0.086901,-0.267605,fast
ID00011637202177653955184,-0.117604,90.151922,-0.734247,0.024285,0.041098,-0.067835,slow
ID00012637202177665765362,-0.234262,103.488901,-0.747992,0.020464,0.078566,-0.117710,fast
...,...,...,...,...,...,...,...
ID00419637202311204720264,-0.049642,65.744402,-0.386749,0.303836,0.044739,-0.039264,slow
ID00421637202311550012437,0.017906,82.794470,0.165532,0.695250,0.043553,0.011246,slow
ID00422637202311677017371,-0.188309,78.850496,-0.812077,0.014341,0.055243,-0.124185,fast
ID00423637202312137826377,-0.221725,73.387770,-0.647303,0.059476,0.098684,-0.157106,fast


In [7]:
# glcm_res = np.load('Data/stepwise_GLCMtop10.npy', allow_pickle= True).item()
# res = np.array(list(glcm_res.keys()))
# swclass=glcm_res[max(res)]
# print(max(res))

In [8]:
jdf = jdf.join(adf).join(GLCMdf).dropna()
jdf

INFO:numexpr.utils:Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


,slope,intercept,r_value,p_value,std_err,nor,decay,emphysema|bottom,emphysema|middle,emphysema|overall,...,intensity_std_std|overall,intensity_std_std|upper,sample_mean_HU_std|bottom,sample_mean_HU_std|middle,sample_mean_HU_std|overall,sample_mean_HU_std|upper,sample_std_HU_std|bottom,sample_std_HU_std|middle,sample_std_HU_std|overall,sample_std_HU_std|upper
ID00007637202177411956430,-0.079696,54.716059,-0.640030,0.063366,0.036162,-0.075740,slow,0.003498,0.012382,0.007578,...,1.236937,0.740513,87.368992,43.382061,98.595418,9.896725,14.606428,22.796989,33.758880,10.328633
ID00009637202177434476278,-0.218565,89.052698,-0.868881,0.002359,0.047065,-0.127626,fast,0.003867,0.011150,0.013633,...,1.294097,1.097691,31.097943,15.872135,27.648544,13.730186,14.902749,12.334904,17.486356,9.156147
ID00010637202177584971671,-0.458238,89.043063,-0.893802,0.001157,0.086901,-0.267605,fast,0.004024,0.044436,0.042186,...,1.105484,0.903177,27.537889,21.054909,38.742991,16.943321,10.973777,9.534408,10.505338,10.386429
ID00012637202177665765362,-0.234262,103.488901,-0.747992,0.020464,0.078566,-0.117710,fast,0.013151,0.034466,0.022931,...,0.801214,0.586670,80.811975,21.452294,68.414512,31.220825,12.227297,10.994380,18.279326,14.603010
ID00014637202177757139317,-0.332042,89.135606,-0.833751,0.005222,0.083111,-0.193707,fast,0.050146,0.031635,0.035613,...,1.159556,0.612074,39.397189,22.797378,42.399736,9.400356,6.227522,12.432716,18.124206,8.298446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID00419637202311204720264,-0.049642,65.744402,-0.386749,0.303836,0.044739,-0.039264,slow,0.065155,0.084721,0.114891,...,1.105966,0.627980,20.607683,13.014961,29.164994,21.090505,16.840391,9.105792,13.765921,7.245936
ID00421637202311550012437,0.017906,82.794470,0.165532,0.695250,0.043553,0.011246,slow,0.023009,0.025897,0.025013,...,1.343634,0.981868,36.307271,24.164298,39.704799,12.027492,17.822981,21.893842,27.971934,12.300506
ID00422637202311677017371,-0.188309,78.850496,-0.812077,0.014341,0.055243,-0.124185,fast,0.028460,0.043371,0.042634,...,1.112423,0.682791,43.569703,27.438575,56.458128,8.595477,4.940117,13.309937,22.770982,8.643956
ID00423637202312137826377,-0.221725,73.387770,-0.647303,0.059476,0.098684,-0.157106,fast,0.028166,0.026525,0.024977,...,1.146747,0.906094,20.802618,26.007278,40.199863,28.150953,4.221707,11.469199,12.841006,18.789490


In [10]:
tr = jdf[abs(jdf['nor'] + 0.09) > 0.01]
print(tr['decay'].value_counts())

slow    92
fast    69
Name: decay, dtype: int64


In [13]:
parameters = {'kernel':['poly'], 'degree':[2,3],'coef0':np.arange(-0.5, 0.5, 0.1),\
                         'C': list(range(1, 5)), 'gamma':np.arange(0.1, 1.5, 0.1)}


### recursively adding features with 100 different initial conditions


In [15]:
best_res = 0#results.best_score_
save_results={}
print(best_res)
save_results = {}
for j in range(100):
    np.random.shuffle(odclass)
    final_cl, final_res = stepwise(add_class=odclass, ini_classes=[], best_res=best_res, refit='AUC')
    save_results[np.round(final_res,3)] = final_cl.copy()
    print(final_res)

In [ ]:
np.max(list(save_results.keys()))

In [ ]:
save_results[0.734]

In [ ]:
#np.save('stepwise100_FVC_auc.npy',save_results)